In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np
from collections import defaultdict

In [2]:
def distance(instance1, instance2, k):
  distance =0;
  mm1 = instance1[0]
  cm1 = instance1[1]
  mm2 = instance2[0]
  cm2 = instance2[1]
  distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
  distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
  distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
  distance-= k
  return distance

In [3]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [4]:
def getResponse(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [5]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [17]:
directory = 'C:/Users/abc/Downloads/Data/genres_original'
f= open("my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break 	
    for file in os.listdir(directory+"/"+folder):	
        try:
            (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
            mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix , covariance , i)
            pickle.dump(feature , f)
        except Exception as e:
            print("Got an exception: ", e, 'in folder: ', folder, ' filename: ', file)
f.close()
        

Got an exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported. in folder:  jazz  filename:  jazz.00054.wav


In [18]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break	

    for x in range(len(dataset)):
        if random.random() <split :			
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])	

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)

In [19]:
predictions=[]
k = 5
for x in range(len(testSet)):
		neighbors = getNeighbors(trainingSet, testSet[x], k)
		result = getResponse(neighbors)
		predictions.append(result)
		print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + repr(accuracy) + '%')

> predicted=3, actual=1
> predicted=10, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=5, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=3, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=4, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=9, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=1, actual=1
> predicted=9, actual=1
> predicted=6, actual=1
> predicted=9, 

In [38]:
results = defaultdict(int)

i=1
for folder in os.listdir(directory):
    results[i]=folder
    i+=1
print(results)

(rate,sig) = wav.read('C:/Users/abc/Downloads/Data/genres_original/blues/blues.00004.wav')
mfcc_feat = mfcc(sig,rate,winlen=0.020,appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix,covariance,0)

prediction = getResponse(getNeighbors(dataset ,feature , 5))
print(prediction)
print(results[prediction])


defaultdict(<class 'int'>, {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'})
1
blues
